In [ ]:
# !pip install ultralytics

In [ ]:
import cv2
from ultralytics.solutions import distance_calculation
from ultralytics import YOLO

model = YOLO("yolov8m.pt")
names = model.model.names

# Video capture
cap = cv2.VideoCapture("people_walking.mp4")
assert cap.isOpened(), "Error reading video file"

# Set new dimensions
new_w, new_h = 640, 360  # Example: Resize to 640x360
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter(
    "distance_calculation.avi",
    cv2.VideoWriter_fourcc(*"mp4v"), 
    fps,
    (new_w, new_h)  # Use the new dimensions
)

# Init distance-calculation obj
dis_obj = distance_calculation.DistanceCalculation()

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    
    # Resize the frame
    im0_resized = cv2.resize(im0, (new_w, new_h))
    
    # Uncomment the following line if distance calculation is needed
    im0_resized = dis_obj.calculate(im0_resized)
    
    # Write resized frame to the output video
    video_writer.write(im0_resized)

cap.release()
video_writer.release()
cv2.destroyAllWindows()


In [ ]:
import math
import cv2
from ultralytics.solutions.solutions import BaseSolution
from ultralytics.utils.plotting import Annotator

class DistanceCalculation(BaseSolution):
    def __init__(self, **kwargs):
        """Initializes the DistanceCalculation class for measuring object distances in video streams."""
        super().__init__(**kwargs)

        # Mouse event information
        self.selected_points = []  # List to store selected points
        self.annotator = None  # Annotator will be initialized in calculate method

    def mouse_event_for_distance(self, event, x, y, flags, param):
        """
        Handles mouse events to select points for distance calculation.
        """
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(self.selected_points) < 4:  # Allow a maximum of 4 points to be selected
                self.selected_points.append((x, y))
                print(f"Point selected: {x}, {y}")

        elif event == cv2.EVENT_RBUTTONDOWN:
            self.selected_points = []  # Reset selection on right click
            print("Selection reset.")

    def calculate(self, im0):
        """
        Processes a video frame and calculates the distance between selected points.
        """
        # Initialize annotator (although we won't use it for circles/lines)
        self.annotator = Annotator(im0)  # Initialize annotator, but we will use OpenCV directly

        # If at least two points are selected, calculate distance between them
        if len(self.selected_points) >= 2:
            for i, (x1, y1) in enumerate(self.selected_points):
                for j, (x2, y2) in enumerate(self.selected_points):
                    if i < j:  # Avoid recalculating the same pair (i, j)
                        # Calculate Euclidean distance between the two points
                        pixels_distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

                        # Round the distance for display
                        distance_str = str(round(pixels_distance, 2))

                        # Annotate the points and the distance on the image using OpenCV
                        # Draw circles on the selected points
                        cv2.circle(im0, (x1, y1), 5, (0, 0, 255), -1)  # Red circle for point 1
                        cv2.circle(im0, (x2, y2), 5, (0, 0, 255), -1)  # Red circle for point 2

                        # Plot the line between points
                        cv2.line(im0, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green line

                        # Midpoint to annotate the distance
                        mid_point = ((x1 + x2) // 2, (y1 + y2) // 2)
                        cv2.putText(im0, distance_str, mid_point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Display the image with annotations
        self.display_output(im0)
        cv2.setMouseCallback("Ultralytics Solutions", self.mouse_event_for_distance)

        return im0  # Return output image for further usage


In [ ]:
import cv2
from ultralytics.solutions import distance_calculation
from ultralytics import YOLO

#image read

# image = cv2.imread("processed_image.jpg")
# Video capture
cap = cv2.VideoCapture(r"c:\Users\abdul\Downloads\How to use Dental X-Ray Film .mp4")
assert cap.isOpened(), "Error reading video file"

# Set new dimensions
new_w, new_h = 640, 360  # Example: Resize to 640x360
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter(
    "distance_calculation.avi",
    cv2.VideoWriter_fourcc(*"mp4v"), 
    fps,
    (new_w, new_h)  # Use the new dimensions
)

# Init distance-calculation obj
dis_obj = DistanceCalculation()

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    
    # Resize the frame
    im0_resized = cv2.resize(im0, (new_w, new_h))
    
    # Uncomment the following line if distance calculation is needed
    im0_resized = dis_obj.calculate(im0_resized)
    
    # Write resized frame to the output video
    video_writer.write(im0_resized)

cap.release()
video_writer.release()
cv2.destroyAllWindows()


In [10]:
import math
import cv2
from ultralytics.solutions.solutions import BaseSolution
from ultralytics.utils.plotting import Annotator

class DistanceCalculation(BaseSolution):
    def __init__(self, **kwargs):
        """Initializes the DistanceCalculation class for measuring object distances in images."""
        super().__init__(**kwargs)
        self.selected_points = []  # List to store selected points

    def mouse_event_for_distance(self, event, x, y, flags, param):
        """
        Handles mouse events to select points for distance calculation.
        """
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(self.selected_points) < 4:  # Allow a maximum of 4 points to be selected
                self.selected_points.append((x, y))
                print(f"Point selected: {x}, {y}")
        elif event == cv2.EVENT_RBUTTONDOWN:
            self.selected_points = []  # Reset selection on right click
            print("Selection reset.")

    def calculate(self, im0):
        """
        Processes an image and calculates the distance between selected points.
        """
        # Create a copy of the original image for drawing
        output_image = im0.copy()

        # If at least two points are selected, calculate distance between them
        if len(self.selected_points) >= 2:
            for i, (x1, y1) in enumerate(self.selected_points):
                for j, (x2, y2) in enumerate(self.selected_points):
                    if i < j:  # Avoid recalculating the same pair (i, j)
                        # Calculate Euclidean distance between the two points
                        pixels_distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

                        # Round the distance for display
                        distance_str = str(round(pixels_distance, 2))

                        # Annotate the points and the distance on the image
                        cv2.circle(output_image, (x1, y1), 3, (0, 0, 255), -1)  # Red circle
                        cv2.circle(output_image, (x2, y2), 3, (0, 0, 255), -1)  # Red circle
                        cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green line
                        mid_point = ((x1 + x2) // 2, (y1 + y2) // 2)
                        cv2.putText(output_image, distance_str, mid_point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        return output_image


def process_image(file_path, output_path):
    """
    Loads an image, allows distance calculations using mouse events, and saves the result.
    """
    im0 = cv2.imread(file_path)
    if im0 is None:
        print("Error: Image not found or could not be read.")
        return

    dis_obj = DistanceCalculation()

    # Create a window
    window_name = "Distance Calculation"
    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, dis_obj.mouse_event_for_distance)

    while True:
        # Show the image with annotations
        im0_resized = dis_obj.calculate(im0)
        cv2.imshow(window_name, im0_resized)

        # Wait for a key press
        key = cv2.waitKey(1) & 0xFF

        if key != 255:  # Debugging: Print any key press for troubleshooting
            print(f"Key pressed: {key}")

        if key == 27:  # Exit on ESC key (ASCII code 27)
            print("ESC key pressed. Exiting.")
            break

    # Save the annotated image
    cv2.imwrite(output_path, dis_obj.calculate(im0))
    print(f"Processed image saved to {output_path}")

    # Close all OpenCV windows
    cv2.destroyAllWindows()


# Example usage
process_image(r"runs\pose\predict18\image0.jpg", "output_image.png")


Ultralytics Solutions:  {'region': None, 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None}
Point selected: 202, 160
Point selected: 164, 303
Point selected: 161, 311
Point selected: 151, 329
Key pressed: 27
ESC key pressed. Exiting.
Processed image saved to output_image.png


In [2]:
pip install opencv-python-headless


Note: you may need to restart the kernel to use updated packages.


In [9]:
import math
import cv2
import numpy as np
from ultralytics import YOLO  # Import the YOLOv8 model

class DistanceCalculation:
    """
    A class for measuring object distances in images by auto-detecting points.
    """
    def __init__(self):
        """Initializes the DistanceCalculation class."""
        self.selected_points = []  # List to store detected points

    def detect_points(self, im0, max_points=4):
        """
        Automatically detects key points in the image using Shi-Tomasi Corner Detection.
        """
        # Convert image to grayscale for corner detection
        gray = cv2.cvtColor(im0, cv2.COLOR_BGR2GRAY)

        # Detect corners using Shi-Tomasi method
        corners = cv2.goodFeaturesToTrack(gray, max_points, 0.01, 10)
        corners = np.int0(corners)  # Convert to integer coordinates

        # Clear existing points and add new detected points
        self.selected_points = [(int(x), int(y)) for [x, y] in corners]
        print(f"Auto-detected points: {self.selected_points}")

    def calculate(self, im0):
        """
        Draws the detected points and calculates distances between them.
        """
        # Create a copy of the image to draw annotations
        output_image = im0.copy()

        # Annotate the detected points and calculate distances
        if len(self.selected_points) >= 2:
            for i, (x1, y1) in enumerate(self.selected_points):
                for j, (x2, y2) in enumerate(self.selected_points):
                    if i < j:  # Only calculate for unique pairs
                        # Calculate Euclidean distance
                        distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

                        # Draw points, line, and distance
                        cv2.circle(output_image, (x1, y1), 5, (0, 0, 255), -1)  # Red point
                        cv2.circle(output_image, (x2, y2), 5, (0, 0, 255), -1)  # Red point
                        cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green line

                        # Annotate the distance at the midpoint
                        mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2
                        cv2.putText(output_image, f"{distance:.2f}", (mid_x, mid_y),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        return output_image


def process_image_with_yolo_v8(file_path, output_path):
    """
    Automatically detects objects using YOLOv8, calculates distances, and saves the result.
    """
    # Load the image
    im0 = cv2.imread(file_path)
    if im0 is None:
        print(f"Error: Could not read the image at {file_path}")
        return

    # Initialize the YOLOv8 model
    model = YOLO(r'file\runs\train\weights\best.pt')  # Load a pre-trained YOLOv8 model (YOLOv8n is a smaller version)

    # Use YOLOv8 to detect objects
    results = model(im0)  # Run detection
    detected_points = []
    print(results)
    # Process the detected bounding boxes (centers of the boxes)
    for box in results.boxes.xyxy:  # Get the coordinates of the bounding boxes
        x1, y1, x2, y2 = box[0]
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        detected_points.append((center_x, center_y))

    # Initialize the distance calculation object
    dis_obj = DistanceCalculation()

    # Update the selected points with detected YOLO points
    dis_obj.selected_points = detected_points
    print(f"YOLOv8 detected points: {dis_obj.selected_points}")

    # Calculate and save the annotated image
    output_image = dis_obj.calculate(im0)
    cv2.imwrite(output_path, output_image)
    print(f"Processed image saved at {output_path}")

    # Display the annotated image
    window_name = "YOLOv8 Object Detection with Distance Calculation"
    cv2.imshow(window_name, output_image)
    print("Press any key to close the window.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Example usage with YOLOv8 model
process_image_with_yolo_v8("runs\pose\predict18\image0.jpg", "output_image_yolov8.png")



0: 640x448 1 Xray_tooth, 79.4ms
Speed: 3.6ms preprocess, 79.4ms inference, 141.3ms postprocess per image at shape (1, 3, 640, 448)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: ultralytics.engine.results.Keypoints object
masks: None
names: {0: 'Xray_tooth'}
obb: None
orig_img: array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  4,   0,   0],
        [  2,   0,   0],
        [  0,   0,   0],
        ...,
        [109, 104, 105],
        [ 74,  73,  69],
  

AttributeError: 'list' object has no attribute 'boxes'

In [ ]:
import cv2

from ultralytics import solutions

cap = cv2.VideoCapture("people_walking.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("distance_calculation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init distance-calculation obj
distance = solutions.DistanceCalculation(model="yolo11n.pt", show=True)

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    im0 = distance.calculate(im0)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="yck6wEmRC5evuqo3P3kA")
project = rf.workspace("abdul-aziz-dgqpw").project("tooth-x-ray-zf9da")
version = project.version(1)
dataset = version.download("yolov8")
                

In [2]:
import torch
import os
from roboflow import Roboflow
from ultralytics import YOLO
from IPython.display import Image,display
import cv2

# import wandb # use for kaggle GPU


In [ ]:
best_model = r'file\runs\train\weights\best.pt'
#best_model = f'{HOME}/train (5)/kaggle/working/runs/detect/train/weights/best.pt'
# last_model = f'{HOME}train (5)\kaggle\working\runs\detect\train\weights\last.pt'
model = YOLO(best_model)
# model = YOLO("yolov8n-pose.pt")
model.predict(
    source=r"1112.png",
    conf=0.25,
    show=True,
    save=True,  # Saves the visualization with skeletons
    device=0,
    show_conf = True, 
)


In [ ]:
import cv2
import math
from ultralytics import YOLO
from IPython.display import Image, display

def calculate_bone_loss(blue, yellow, mint, red):
    """
    Calculate the percentage of radiographic bone loss.

    Parameters:
    blue (tuple): Coordinates of the CEJ (Cementoenamel Junction) (x, y).
    yellow (tuple): Coordinates of the existing bone level (x, y).
    mint (tuple): Coordinates of the bone crest level (x, y).
    red (tuple): Coordinates of the root apex (x, y).

    Returns:
    float: Percentage of bone loss.
    """
    # Calculate Euclidean distances
    distance_A = math.sqrt((mint[0] - yellow[0])**2 + (mint[1] - yellow[1])**2)
    distance_B = math.sqrt((mint[0] - red[0])**2 + (mint[1] - red[1])**2)
    
    if distance_B == 0:
        raise ValueError("Distance_B cannot be zero (division by zero).")
    
    # Calculate percentage
    percentage_bone_loss = (distance_A / distance_B) * 100
    return percentage_bone_loss

def process_image(image_path, yolo_model):
    """
    Detect key points in the image and calculate bone loss.

    Parameters:
    image_path (str): Path to the input image.
    yolo_model: Pre-trained YOLO model.

    Returns:
    float: Percentage of bone loss.
    """
    # Read the input image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at {image_path}")

    # Perform YOLO inference to detect points
    results = yolo_model.predict(source=image, save=True, save_txt=False)  # YOLO inference

    print (results)


    # Initialize key point coordinates
    blue, yellow, mint, red = None, None, None, None

    # Extract detections from YOLO results
    detections = results[0].boxes
    for box in detections:
        # Calculate center of bounding box
        x_center = int((box.xyxy[0][0] + box.xyxy[0][2]) / 2)
        y_center = int((box.xyxy[0][1] + box.xyxy[0][3]) / 2)
        print(x_center)

        # Get the class label (0=Blue, 1=Yellow, 2=Mint, 3=Red)
        label = int(box.cls[0])

        # Assign coordinates based on label
        if label == 0:  # Blue (CEJ)
            blue = (x_center, y_center)
        elif label == 1:  # Yellow (Existing bone level)
            yellow = (x_center, y_center)
        elif label == 2:  # Mint (Bone crest level)
            mint = (x_center, y_center)
        elif label == 3:  # Red (Root apex)
            red = (x_center, y_center)

    # Ensure all key points are detected
    if blue is None or yellow is None or mint is None or red is None:
        raise ValueError("Not all key points were detected in the image.")

    # Calculate bone loss percentage
    percentage_bone_loss = calculate_bone_loss(blue, yellow, mint, red)

    # Visualize the detections on the image
    cv2.circle(image, blue, 5, (255, 0, 0), -1)    # Blue dot (CEJ)
    cv2.circle(image, yellow, 5, (0, 255, 255), -1)  # Yellow dot (Existing bone level)
    cv2.circle(image, mint, 5, (0, 255, 0), -1)     # Mint dot (Bone crest level)
    cv2.circle(image, red, 5, (0, 0, 255), -1)      # Red dot (Root apex)

    # Save and display the output image
    output_path = "output_image_with_points.jpg"
    cv2.imwrite(output_path, image)
    display(Image(filename=output_path))

    return percentage_bone_loss

# Example usage
image_path = "1112.png"  # Replace with your input image path
yolo_model = YOLO(r'file\runs\train\weights\best.pt')  # Load the trained YOLO model

# Process the image and calculate bone loss
try:
    bone_loss_percentage = process_image(image_path, yolo_model)
    print(f"The percentage of bone loss is: {bone_loss_percentage:.2f}%")
except Exception as e:
    print(f"Error: {e}")



0: 640x448 2 Xray_tooths, 49.4ms
Speed: 0.0ms preprocess, 49.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 448)
Results saved to runs\pose\predict18
129
53
Error: Not all key points were detected in the image.
